# Inference in Bayesian networks

We can implement exact inference using factors. Recall that factors represent discrete multivariate distributions. We use the following three operations on factors to achieve this:

#### 1. Factor product
We use the factor product to combine two factors to produce a larger factor whose scope is the combined scope of the input factors. If we have $φ(X, Y)$ and $ψ(Y, Z)$, then $φ · ψ$ will be over $X$, $Y$, and $Z$ with $(φ · ψ)(x, y, z) = φ(x, y)ψ(y, z)$. 

#### 2. Factor marginalization
We use factor marginalization to sum out a particular variable from the entire factor table, removing it from the resulting scope.

#### 3. Factor conditioning
We use factor conditioning with respect to some evidence to remove any rows in the table inconsistent with that evidence.

### Let's implement the code from last notebook (`1. Representations`) so we can reuse it

In [1]:
struct Variable
    name::Symbol
    m::Int
end

const Assignment = Dict{Symbol, Int}
const FactorTable = Dict{Assignment, Float64}

struct Factor
    vars::Vector{Variable}
    table::FactorTable
end

Base.Dict{Symbol, T}(a::NamedTuple) where T = 
    Dict{Symbol, T}( k=>v for (k,v) in zip(keys(a), values(a)) )

Base.convert(::Type{Dict{Symbol, T}}, a::NamedTuple) where T = 
    Dict{Symbol, T}(a)

Base.isequal(a::Dict{Symbol, T}, b::NamedTuple) where T = 
    length(a) == length(b) &&
    all(a[k] == v for (k,v) in zip(keys(b), values(b)))

variablenames(ϕ::Factor) = [var.name for var in ϕ.vars]

select(a::Assignment, varnames::Vector{Symbol}) = 
    Assignment( n => a[n] for n in varnames )

select (generic function with 1 method)

In [2]:
import Base.Iterators: product

function assignments(vars::AbstractVector{Variable})
    names = [var.name for var in vars]
    matrix_of_assignments = [Assignment(name => p for (name, p) in zip(names, p)) 
        for p in product((1:var.m for var in vars)...)]
    vec(matrix_of_assignments)
end

assignments (generic function with 1 method)

In [3]:
using LightGraphs

function normalize!(ϕ::Factor)
    z = sum(p for (a,p) in ϕ.table)
    for (a,p) in ϕ.table
        ϕ.table[a] = p/z
    end
    ϕ
end

struct BayesianNetwork
    vars::Vector{Variable}
    factors::Vector{Factor}
    graph::SimpleDiGraph{Int64}
end

In [27]:
# ?merge

### 1. Factor product

In [5]:
function Base.:*(ϕ::Factor, ψ::Factor)
    ϕ_names = variablenames(ϕ)
    ψ_names = variablenames(ψ)
    ψ_only = setdiff(ψ.vars, ϕ.vars)
    final_table = FactorTable()
    for (ϕ_assignment, ϕ_prob) in ϕ.table
        for ψ_only_assignment in assignments(ψ_only)
            complete_assignment = merge(ϕ_assignment, ψ_only_assignment)
            ψ_assignment = select(complete_assignment, ψ_names)
            final_table[complete_assignment] = ϕ_prob * get(ψ.table, ψ_assignment, 0.0)
        end
    end
    vars = vcat(ϕ.vars, ψ_only)
    Factor(vars, final_table)
end

### 2. Factor marginalization

In [6]:
function marginalize(ϕ::Factor, name::Symbol)
    final_table = FactorTable()
    for (assignment, prob) in ϕ.table
        assignment_without_name = delete!(copy(assignment), name)
        final_table[assignment_without_name] = get(final_table, assignment_without_name, 0.0) + prob
    end
    final_variables = filter(v -> v.name != name, ϕ.vars)
    Factor(final_variables, final_table)
end

marginalize (generic function with 1 method)

### 3. Factor conditioning

Two methods for factor conditioning given some evidence. 

i) The first takes a factor $φ$ and returns a new factor whose table entries are consistent with the variable named `name` having value `value`. 

ii) The second takes a factor $φ$ and applies evidence in the form of a named tuple. The `in_scope` method returns `true` if a variable named `name` is within the scope of the factor $φ$.

In [62]:
in_scope(name::Symbol, ϕ::Factor) = any(name == v.name for v in ϕ.vars)

function condition(ϕ::Factor, name::Symbol, value)
    if !in_scope(name, ϕ)
        return ϕ
    end
    final_table = FactorTable()
    for (assignment, prob) in ϕ.table
        if assignment[name] == value
            final_table[delete!(copy(assignment), name)] = prob
        end
    end
    final_variables = filter(v -> v.name != name, ϕ.vars)
    Factor(final_variables, final_table)
end

function condition(ϕ::Factor, evidence)
    for (name, value) in pairs(evidence)
        ϕ = condition(ϕ, name, value)
    end
    ϕ
end

condition (generic function with 2 methods)

In [28]:
# ?pairs

### Exact inference

A naive exact inference algorithm for a discrete Bayesian network `bn`, which takes as input a set of query variable names `query`, and `evidence` associating values with observed variables. 

The algorithm computes a joint distribution over the query variables in the form of a factor. 

> We introduce the `ExactInference` type to allow for `infer` to be called with different inference methods, as shall be seen in the rest of this chapter.

Notice how `ExactInference` parameter is not used in the function definition. It is solely used for multiple dispatch.

In [9]:
struct ExactInference end

In [10]:
function infer(::ExactInference, bn, query, evidence)
    ϕ = prod(bn.factors)
    ϕ = condition(ϕ, evidence)
    for name in setdiff(variablenames(ϕ), query)
        ϕ = marginalize(ϕ, name)
    end
    normalize!(ϕ)
end

infer (generic function with 1 method)

### Let's test on the example given in the book

In [32]:
X = Variable(:x, 2)
Y = Variable(:y, 2)
Z = Variable(:z, 2)

ϕ1 = Factor([X,Y], FactorTable(
        (x=1, y=1) => 0.3,
        (x=1, y=2) => 0.4,
        (x=2, y=1) => 0.2,
        (x=2, y=2) => 0.1,
        ))

ϕ2 = Factor([Y,Z], FactorTable(
        (y=1, z=1) => 0.2,
        (y=1, z=2) => 0.0,
        (y=2, z=1) => 0.3,
        (y=2, z=2) => 0.5,
        ));

#### Test factor product

In [33]:
result = ϕ1 * ϕ2
println(result.vars)
result.table

Variable[Variable(:x, 2), Variable(:y, 2), Variable(:z, 2)]


Dict{Dict{Symbol, Int64}, Float64} with 8 entries:
  Dict(:y=>1, :z=>1, :x=>1) => 0.06
  Dict(:y=>1, :z=>1, :x=>2) => 0.04
  Dict(:y=>1, :z=>2, :x=>1) => 0.0
  Dict(:y=>2, :z=>2, :x=>1) => 0.2
  Dict(:y=>2, :z=>1, :x=>2) => 0.03
  Dict(:y=>1, :z=>2, :x=>2) => 0.0
  Dict(:y=>2, :z=>2, :x=>2) => 0.05
  Dict(:y=>2, :z=>1, :x=>1) => 0.12

#### Test factor marginalization

In [63]:
X = Variable(:x, 2)
Y = Variable(:y, 2)
Z = Variable(:z, 2)

ϕ = Factor([X,Y,Z], FactorTable(
        (x=1, y=1, z=1) => 0.08,
        (x=1, y=1, z=2) => 0.31,
        (x=1, y=2, z=1) => 0.09,
        (x=1, y=2, z=2) => 0.37,
        (x=2, y=1, z=1) => 0.01,
        (x=2, y=1, z=2) => 0.05,
        (x=2, y=2, z=1) => 0.02,
        (x=2, y=2, z=2) => 0.07,
        ));

In [64]:
result = marginalize(ϕ, Y.name)
println(result.vars)
result.table

Variable[Variable(:x, 2), Variable(:z, 2)]


Dict{Dict{Symbol, Int64}, Float64} with 4 entries:
  Dict(:z=>2, :x=>2) => 0.12
  Dict(:z=>2, :x=>1) => 0.68
  Dict(:z=>1, :x=>1) => 0.17
  Dict(:z=>1, :x=>2) => 0.03

#### Test factor conditioning

In [65]:
result = condition(ϕ, :y, 2.0)
println(result.vars)
result.table

Variable[Variable(:x, 2), Variable(:z, 2)]


Dict{Dict{Symbol, Int64}, Float64} with 4 entries:
  Dict(:z=>2, :x=>2) => 0.07
  Dict(:z=>2, :x=>1) => 0.37
  Dict(:z=>1, :x=>1) => 0.09
  Dict(:z=>1, :x=>2) => 0.02

In [66]:
result = condition(ϕ, (y=2,x=1))
println(result.vars)
result.table

Variable[Variable(:z, 2)]


Dict{Dict{Symbol, Int64}, Float64} with 2 entries:
  Dict(:z=>1) => 0.09
  Dict(:z=>2) => 0.37

In [67]:
condition(ϕ, (g=2,x=1))

Factor(Variable[Variable(:y, 2), Variable(:z, 2)], Dict(Dict(:y => 2, :z => 2) => 0.37, Dict(:y => 1, :z => 1) => 0.08, Dict(:y => 1, :z => 2) => 0.31, Dict(:y => 2, :z => 1) => 0.09))

### Inference in Naive Bayes models

![](./assets/ex_3.3.png)

> We have to specify the prior $P(C)$ and the class-conditional distributions $P(O_i|C)$

> Our classification task involves computing the conditional probability $P(c | o_{1:n})$

$$P(c | o_{1:n}) = \frac{P(c, o_{1:n})}{P(o_{1:n})}$$

We can compute the denominator by marginalizing the joint distribution. The denominator is not a function of $C$ and can therefore be treated as a constant. Thus we can write

$$P(c | o_{1:n}) \propto P(c, o_{1:n})$$

### Sum-product variable elimination

Consider the below figure:

![](./assets/ex_2.5.png)

Let's say we want to compute the distribution $P(B | d^1, c^1)$

The conditional probability distributions associated with the nodes in the network can be __represented__ by the following factors:

$$ϕ_1(B), ϕ_2(S), ϕ_3(E,B,S), ϕ_4(D,E), ϕ_5(C,E)$$

Because $D$ and $C$ are observed variables, the last two factors can be replaced with $φ_6(E)$ and $φ_7(E)$ by setting the evidence $D = 1$ and $C = 1$.

We eliminate $E$ then $S$:

$$φ_8(B,S) = ∑_e φ_3(e,B,S)φ_6(e)φ_7(e)$$
$$φ_9(B) = ∑φ_2(s)φ_8(B,s)$$

Finally we take the product of $φ_9(B)$ and $φ_1(B)$ and normalize it to get $P(B|d^1,c^1)$

__Previous way__: Taking the product of all factors then marginalizing

$$ P(B | d^1,c^1) ∝ ∑_s ∑_e φ_1(B)φ_2(s)φ_3(e | B,s)φ_4(d^1 | e)φ_5(c^1 | e) $$


__Sum product way__:

$$ P(B | d^1, c^1) ∝ φ_1(B) ∑_s( φ_2(s) ∑_e (φ_3(e | B, s) φ_4(d^1 | e) φ_5(c^1 | e)) )$$

In [74]:
struct VariableElimination
    ordering # array of variable indices
end

function infer(M::VariableElimination, bn, query, evidence)
    ϕs_after_evidence = [condition(ϕ, evidence) for ϕ in bn.factors]
    for index in M.ordering
        name = bn.vars[index].name
        if name ∉ query
            indices_containing_name = findall(ϕ -> in_scope(name, ϕ), ϕs_after_evidence)
            if !isempty(indices_containing_name)
                ϕ = prod(ϕs_after_evidence[indices_containing_name])
                deleteat!(ϕs_after_evidence, indices_containing_name)
                ϕ_after_marginalizing = marginalize(ϕ, name)
                push!(ϕs_after_evidence, ϕ_after_marginalizing)
            end
        end
    end
    normalize!(prod(ϕs_after_evidence))
end
        

infer (generic function with 2 methods)

In [75]:
# ?deleteat!

In [76]:
# ?findall

### Let's test the new `infer` function

In [77]:
B = Variable(:b, 2)
S = Variable(:s, 2)
E = Variable(:e, 2)
D = Variable(:d, 2)
C = Variable(:c, 2)

vars = [B,S,E,D,C]

factorB = Factor([B], FactorTable((b=1,)=>0.99, (b=2,)=>0.01))
factorS = Factor([S], FactorTable((s=1,)=>0.98, (s=2,)=>0.02))
factorE = Factor([B,S,E], FactorTable(
        (e=1,b=1,s=1) => 0.90,
        (e=1,b=1,s=2) => 0.04,
        (e=1,b=2,s=1) => 0.05,
        (e=1,b=2,s=2) => 0.01,
        (e=2,b=1,s=1) => 0.10,
        (e=2,b=1,s=2) => 0.96,
        (e=2,b=2,s=1) => 0.95,
        (e=2,b=2,s=2) => 0.99,
        ))
factorD = Factor([D,E], FactorTable(
        (d=1,e=1) => 0.96,
        (d=1,e=2) => 0.03,
        (d=2,e=1) => 0.04,
        (d=2,e=2) => 0.97,
        ))
factorC = Factor([C,E], FactorTable(
        (c=1,e=1) => 0.98,
        (c=1,e=2) => 0.01,
        (c=2,e=1) => 0.02,
        (c=2,e=2) => 0.99,
        ))

graph = SimpleDiGraph(5)

add_edge!(graph, 1, 3)
add_edge!(graph, 2, 3)
add_edge!(graph, 3, 4)
add_edge!(graph, 3, 5)

bn = BayesianNetwork(vars, [factorB, factorS, factorE, factorD, factorC], graph);

In [80]:
M = VariableElimination([3,2]) # eliminate E then S
evidence = (d=2,c=2)
factor = infer(M, bn, [B.name], evidence)
println(factor.vars)
factor.table

Variable[Variable(:b, 2)]


Dict{Dict{Symbol, Int64}, Float64} with 2 entries:
  Dict(:b=>2) => 0.0753055
  Dict(:b=>1) => 0.924695

### Direct sampling

Idea: Random samples from the joint distribution are used to arrive at a probability estimate.

Suppose we want to infer $P(b^1 | d^1, c^1)$ from a set of `n` samples from the joint distribution `P(b,s,e,d,c)`. The direct sample estimate is:

![](./assets/direct_sampling_1.png)

The first step involves finding a topological sort of the nodes in the Bayesian network. A topological sort always exists, but it may not be unique.

Once we have a topological sort, we can begin sampling from the conditional probability distributions.

In [81]:
function Base.rand(ϕ::Factor)
    # the idea is to return a random assignment from ϕ
    # the assignment is chosen based on the `random_number`
    # note that this will result in assignments with high probability being returned more often
    random_number = rand()
    total_prob = sum(values(ϕ.table))
    total = 0.0
    for (assignment, prob) in ϕ.table
        total += prob
        if total >= random_number
            return assignment
        end
    end
    return Assignment()
end

In [83]:
bn.graph

{5, 4} directed simple Int64 graph

In [86]:
topological_sort_by_dfs(bn.graph)

5-element Vector{Int64}:
 2
 1
 3
 5
 4

In [114]:
function Base.rand(bn::BayesianNetwork)
    output = Assignment()
    for i in topological_sort_by_dfs(bn.graph)
        variable, ϕ = bn.vars[i].name, bn.factors[i]
        random_assignment_from_factor = rand(condition(ϕ, output))
        output[variable] = random_assignment_from_factor[variable]
    end
    return output
end

In [90]:
struct DirectSampling
    m #number of samples
end

function infer(M::DirectSampling, bn, query, evidence)
    table = FactorTable()
    for i in 1:(M.m)
        assignment = rand(bn)
        if all(assignment[k] == v for (k,v) in pairs(evidence))
            sub_assignment = select(assignment, query)
            table[sub_assignment] = get(table, sub_assignment, 0) + 1
        end
    end
    vars = filter(v -> v.name ∈ query, bn.vars)
    return normalize!(Factor(vars, table))
end

infer (generic function with 3 methods)

In [118]:
infer(DirectSampling(100), bn, [B.name], (d=2, c=2))

Factor(Variable[Variable(:b, 2)], Dict(Dict(:b => 2) => 0.16666666666666666, Dict(:b => 1) => 0.8333333333333334))

In [119]:
infer(DirectSampling(1000), bn, [B.name], (d=2, c=2))

Factor(Variable[Variable(:b, 2)], Dict(Dict(:b => 2) => 0.08064516129032258, Dict(:b => 1) => 0.9193548387096774))

In [120]:
infer(DirectSampling(10000), bn, [B.name], (d=2, c=2))

Factor(Variable[Variable(:b, 2)], Dict(Dict(:b => 2) => 0.08067226890756303, Dict(:b => 1) => 0.9193277310924369))

In [122]:
infer(DirectSampling(100000), bn, [B.name], (d=2, c=2))

Factor(Variable[Variable(:b, 2)], Dict(Dict(:b => 2) => 0.0759514571121935, Dict(:b => 1) => 0.9240485428878065))